In [2]:
%pwd


'd:\\GEN AI\\End-to-End-Medical-Chatbot\\research'

In [3]:
import os
os.chdir("d:\\GEN AI\\End-to-End-Medical-Chatbot")
%pwd

'd:\\GEN AI\\End-to-End-Medical-Chatbot'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

In [5]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [6]:
extracted_data=load_pdf_file(data='Data/')

In [7]:
# extracted_data

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

# Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

embeddings = download_hugging_face_embeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

d:\miniconda\envs\medibot\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Microsoft\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [16]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [17]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [21]:
from langchain.vectorstores import Pinecone

# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = Pinecone.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


In [22]:
from langchain.vectorstores import Pinecone

# Load the existing Pinecone index
docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)


In [27]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

retriever.invoke("how to increase bone density")


[Document(metadata={'page': 554.0, 'source': 'Data\\Medical_book.pdf'}, page_content='osteoporosis is already evident. These patients should\nhave a repeat bone density scan every year or two.\nResources\nBOOKS\nBonnick, Sydney Lou. The Osteoporosis Handbook. Taylor\nPublishing, 1994.\nBrown, Susan E. Better Bones, Better Body: A Comprehensive\nSelf-Help Program for Preventing, Halting and Overcoming\nOsteoporosis.New Canaan, CT: Keats Publishing, 1996.\nPERIODICALS\nBilger, Burkhard. “Bone Medicine.”Health, May/June 1996,\n125-128.\n“Bone Density Testing: Should You Be Checked?”Mayo Clinic'),
 Document(metadata={'page': 556.0, 'source': 'Data\\Medical_book.pdf'}, page_content='vitamin D. Dairy products and fish such as salmon, sar-\ndines and tuna are good sources of both calcium and vita-\nmin D. People who are taking bone disorder drugs for\nosteoporosis and who do not get enough of these nutri-\nents in their diets should check with their physicians\nabout taking supplements. Other

In [30]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0, max_tokens=500)

In [33]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [34]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [39]:
response = rag_chain.invoke({"input": "how to control diabetes"})
print(response["answer"])



The American Diabetes Association recommends that people with diabetes keep their blood glucose levels as close to normal as possible. This can be achieved by regularly testing blood sugar levels at home. The ADA also recommends that each blood sugar test using plasma requires a 5 mL blood sample, which can be obtained by a healthcare worker tying a tight band on the person's upper arm and locating a vein. Medications such as sulfonylureas and alpha-glucosidase inhibitors can also help control diabetes by increasing insulin release and delaying the conversion of carbohydrates to glucose, respectively.
